<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_CS6910/blob/main/Assignment3/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Model
from keras.layers import SimpleRNN, LSTM, GRU, Dense, Dropout, TimeDistributed, Lambda, Activation, Reshape,\
Softmax, Multiply, AdditiveAttention, Concatenate, Add, RepeatVector
import tarfile
import random
from keras.optimizers import RMSprop, Adam, SGD
from keras.utils import to_categorical

In [ ]:
!pip install wandb
!wandb login
import wandb
from wandb.keras import WandbCallback
wandb.init(project="CS6910-assg3", entity="narendv")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Examples

Loading Data

In [ ]:
my_tar = tarfile.open('/content/drive/MyDrive/Dakshina dataset/dakshina_dataset_v1.0.tar')
my_tar.extractall('/content/drive/MyDrive/Dakshina dataset') # specify which folder to extract to
my_tar.close()

Examples

In [ ]:
import csv
hi_to_eng = open("/content/drive/MyDrive/Dakshina dataset/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv")
read_tsv = csv.reader(hi_to_eng, delimiter="\t")
i=0
for row in read_tsv:
  i +=1
  print(row)
  if i==30:
    break
hi_to_eng.close()

# Loading datasets

In [3]:
def load_data(data_path, data):
  input_texts=[]
  target_texts = []
  global tk_input, tk_target, num_input_tokens, num_target_tokens
  
  with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
  for line in lines[: (len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
     
  if (data == "train"):
    tk_input = keras.preprocessing.text.Tokenizer(num_words=None, char_level=True)
    tk_input.fit_on_texts(input_texts)
    enc_input = tk_input.texts_to_sequences(input_texts)
    enc_input = tf.keras.preprocessing.sequence.pad_sequences(enc_input, padding='post')

    tk_target = keras.preprocessing.text.Tokenizer(num_words=None, char_level=True)
    tk_target.fit_on_texts(target_texts)
    dec_target = tk_target.texts_to_sequences(target_texts)
    dec_target = tf.keras.preprocessing.sequence.pad_sequences(dec_target, padding='post')

    dec_input = np.zeros(dec_target.shape)
    dec_input[:,:(dec_target.shape[1]-1)] = dec_target[:,1:]
  
  else:
    enc_input = tk_input.texts_to_sequences(input_texts)
    enc_input = tf.keras.preprocessing.sequence.pad_sequences(enc_input, padding='post', maxlen = max_input_length)

    dec_target = tk_target.texts_to_sequences(target_texts)
    dec_target = tf.keras.preprocessing.sequence.pad_sequences(dec_target, padding='post', maxlen = max_target_length)

    dec_input = np.zeros(dec_target.shape)
    dec_input[:,:(dec_target.shape[1]-1)] = dec_target[:,1:]
  
  return enc_input, dec_target, dec_input

In [4]:
# Number of samples to train on.
# Path to the data txt file on disk.
data_path_train = "/content/drive/MyDrive/Dakshina dataset/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
data_path_dev = '/content/drive/MyDrive/Dakshina dataset/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv'

In [5]:
global enc_input_train, dec_input_train, dec_target_train, enc_input_dev, dec_input_dev, dec_target_dev
global tk_input, tk_target
global input_token_train, output_token_train, max_input_length, max_target_length
global y_true_dev, y_true_train

batch_size = 256

enc_input_train, dec_input_train, dec_target_train = load_data(data_path_train,'train')

max_input_length = enc_input_train.shape[1]
max_target_length = dec_target_train.shape[1]

num_input_tokens = len(tk_input.word_index)+1
num_target_tokens = len(tk_target.word_index)+1

enc_input_dev, dec_input_dev, dec_target_dev = load_data(data_path_dev,'dev')

In [7]:
y_true_dev = np.zeros((dec_target_dev.shape[0],dec_target_dev.shape[1], num_target_tokens))
for i in range(0,dec_target_dev.shape[0]):
  y_true_dev[i] = to_categorical(dec_target_dev[i], num_classes = num_target_tokens)

y_true_train = np.zeros((dec_target_train.shape[0],dec_target_train.shape[1], num_target_tokens))
for i in range(0,dec_target_train.shape[0]):
  y_true_train[i] = to_categorical(dec_target_train[i], num_classes = num_target_tokens)

# Defining functions

In [8]:
def encoder(enc_out,enc_block, latent_dim, enc_layers):
  states = []
  for i in range(0,enc_layers):
    if enc_block =='LSTM':
      enc_out ,h ,c = LSTM(latent_dim, return_state=True, return_sequences=True, name='encoder'+str(i))(enc_out)
      s = [h,c]
      states.append(s)
    elif enc_block =='GRU':
      enc_out ,s = GRU(latent_dim, return_state=True, return_sequences=True, name='encoder'+str(i))(enc_out)
      states.append(s)
    else:
      enc_out ,s = SimpleRNN(latent_dim, return_state=True, return_sequences=True, name='encoder'+str(i))(enc_out)
      states.append(s)
    
  return enc_out,states

In [9]:
def b_attention(latent_dim):

  query_in = keras.Input((latent_dim,))
  values_in = keras.Input((max_input_length,latent_dim))
  query = Reshape((1,latent_dim), name='query_reshape')(query_in)
  score_unit2 = Dense(num_target_tokens, name='score_unitq')(query)
  score_unit1 = Dense(num_target_tokens, name='score_unitv')(values_in)
  score_unit12 = Add()([score_unit1, score_unit2])
  score_unit12 = Activation('tanh')(score_unit12)
  score = Dense(1)(score_unit12)
  attention_weights = Softmax(axis=1)(score)
  context_vector = Multiply()([attention_weights,values_in])
  context_vector = Lambda(lambda x: tf.reduce_sum(x,axis=1))(context_vector)
  context_vector = RepeatVector(max_target_length)(context_vector)

  model = Model(inputs = [query_in,values_in], outputs = [context_vector, attention_weights], name='attention')
  return model

In [10]:
def decoder(X_input_dec, hs_init, latent_dim, dec_block, dec_layers, dense_no, drop_no):
  out_dec = X_input_dec
  hs = hs_init
  for i in range (0,dec_layers):
    if dec_block == 'LSTM':
      out_dec, h, s = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder'+str(i))(out_dec, initial_state = hs[i])
      # hs = [h,s]
    elif dec_block == 'GRU':
      out_dec, _ = GRU(latent_dim, return_sequences=True, return_state=True, name='decoder'+str(i))(out_dec, initial_state = hs[i])
    else:
      out_dec, _ = SimpleRNN(latent_dim, return_sequences=True, return_state=True, name='decoder'+str(i))(out_dec, initial_state = hs[i])

  out_dec = TimeDistributed(Dense(num_target_tokens, activation="softmax", name = 'output'), name = 'dense')(out_dec)
  return out_dec

# Rest of the code

In [11]:
def RNN_model(embedding_no, enc_block, latent_dim, dec_block, dense_no, drop_no, enc_layers, dec_layers):
  X_input_enc = keras.Input(shape=(max_input_length,))
  X_input_dec = keras.Input(shape=(max_target_length,))
  X_enc = tf.keras.layers.Embedding(num_input_tokens, embedding_no, name = 'embedding_enc')(X_input_enc)
  X_dec = tf.keras.layers.Embedding(num_target_tokens, embedding_no, name = 'embedding_dec')(X_input_dec)
  _,s_init = encoder(X_enc,enc_block, latent_dim, enc_layers)
  out_dec = decoder(X_dec,s_init, latent_dim, dec_block, dec_layers, dense_no, drop_no)
  model = Model(inputs = [X_input_enc,X_input_dec], outputs = out_dec, name = 'train_model')
  return model

In [12]:
def RNN_w_att(embedding_no, enc_block, latent_dim, dec_block, dense_no, drop_no, enc_layers, dec_layers):
  X_input_enc = keras.Input(shape=(max_input_length,))
  X_input_dec = keras.Input(shape=(max_target_length,))

  X_enc = tf.keras.layers.Embedding(num_input_tokens, embedding_no, name = 'embedding_enc')(X_input_enc)
  X_dec = tf.keras.layers.Embedding(num_target_tokens, embedding_no, name = 'embedding_dec')(X_input_dec)

  values,s_init = encoder(X_enc,enc_block, latent_dim, enc_layers)
  context_vector, weights = b_attention(latent_dim)([s_init[len(s_init)-1][0], values])
  X_dec = Concatenate(name = 'concatenate')([context_vector,X_dec])

  out_dec = decoder(X_dec,s_init, latent_dim, dec_block, dec_layers, dense_no, drop_no)
  model = Model(inputs = [X_input_enc,X_input_dec], outputs = out_dec, name = 'train_model')

  return model

In [ ]:
model = RNN_model(256, 'LSTM', 512, 'LSTM', 256, 0, 2, 2)
keras.utils.plot_model(model,show_shapes=True, rankdir='TB')

In [15]:
model.compile(optimizer=Adam(1e-2), loss = 'categorical_crossentropy', metrics=['accuracy'])#, word_accuracy])
model.fit([enc_input_train,dec_input_train], y_true_train, 
          batch_size = 128,epochs=10, validation_data=([enc_input_dev,dec_input_dev], y_true_dev))

In [16]:
def word_accuracy(y_true,y_pred):
  acc = tf.cast(tf.math.reduce_all(tf.math.reduce_all(tf.equal(y_pred,y_true),-1),-1),dtype=tf.int32)
  return tf.reduce_sum(acc)/tf.shape(y_true)[0]

In [17]:
def enc_inference(model,enc_layers):
  X_enc = model.input[0]
  X_in_enc = model.get_layer('embedding_enc')(X_enc)
  enc_outputs = X_in_enc
  states = []
  for i in range(0,enc_layers):
    enc_outputs, h_enc, s_enc = model.get_layer('encoder'+str(i))(enc_outputs)
    states.append([h_enc, s_enc])
  enc_model = keras.Model(inputs = X_enc, outputs = states)
  return enc_model

def dec_inference(model, latent_dim, dec_layers):
  dec_inputs = model.input[1]
  dec_outputs = model.get_layer('embedding_dec')(dec_inputs)
  hs_inputs = []
  for i in range(0,dec_layers):
    h = keras.Input(shape=(latent_dim,))
    c = keras.Input(shape=(latent_dim,))
    hs_inputs.append([h,c])
  hs_outputs = []
  for i in range(0,dec_layers):
    dec_outputs, h_dec, s_dec = model.get_layer('decoder'+str(i))(dec_outputs, initial_state = hs_inputs[i])
    hs_outputs.append([h_dec,s_dec])
  dec_dense = model.get_layer('dense')
  dec_outputs = dec_dense(dec_outputs)
  dec_model = keras.Model(inputs = [dec_inputs] + hs_inputs, outputs = [dec_outputs] + hs_outputs)  
  return dec_model

In [18]:
enc_inf = enc_inference(model, 2)
dec_inf = dec_inference(model, 512, 2)

In [38]:
reverse_input_char_index = dict((i, char) for char, i in tk_input.word_index.items())
reverse_target_char_index = dict((i, char) for char, i in tk_target.word_index.items())

In [55]:
def decode_sequence(input_seq):
  states = enc_inf.predict(input_seq)
  target_seq = np.zeros((input_seq.shape[0],1))
  outputs = [""]*input_seq.shape[0]
  target_seq[:,0] = tk_target.word_index["\t"]
  stop_key = tk_target.word_index["\n"]
  outputs = [""]*input_seq.shape[0]
  for i in range(input_seq.shape[0]):
      outputs[i] = outputs[i]+ str(reverse_target_char_index[target_seq[i][0]])
  for i in range(max_target_length):
    output_tokens, h, c = dec_inf.predict([target_seq] + states)
    x = np.argmax(output_tokens, axis=-1)
    target_seq = x
    for j in range(input_seq.shape[0]):
      outputs[j] = outputs[j]+ str(reverse_target_char_index[target_seq[j][0]])
    states = [h,c]
  return outputs

In [57]:
input_seq = enc_input_dev[0:2,:]
pred = decode_sequence(input_seq)
print (pred)

In [59]:
def predict(model,latent_dim,enc_layers, dec_layers):
  enc_block = enc_inference(model, enc_layers)
  dec_block = dec_inference(model,latent_dim, dec_layers)
  # keras.utils.plot_model(dec_block,show_shapes=True)
  #input_seq = tf.expand_dims(enc_input_dev[10],0)
  input_seq = tf.expand_dims(enc_input_dev[0], 0)
  decoded_word = decode_sequence(input_seq, enc_block, dec_block)

  return decoded_word

In [ ]:
import pprint

In [ ]:
sweep_config = {
    'method': 'grid'
    }

parameters_dict = {
    'encoder_block':{
        'values': ['LSTM']
      },
    # 'encoder_layers': {
    #     'values': [1,2,3]
    #   },
    'decoder_block':{
      'values' : ['LSTM']
      },
    # 'decoder_layers': {
    #     'values': [1,2,3]
    #   },
    'latent_dimension': {
        'values': [256]
      },
    'hidden_layer': {
        'values': [128]
      },
    'dropout': {
        'values': [0]
      },
    'epochs' : {
        'values' : [60]
    }
}

sweep_config['parameters'] = parameters_dict
pprint.pprint(sweep_config)

def training_sweep(config=None):
    with wandb.init(config=config):
        print(input_token_train)
        config = wandb.config
        print(output_token_train)
        model = RNN_model(input_token_train, config.encoder_block, config.latent_dimension, output_token_train, config.decoder_block,  config.hidden_layer, config.dropout)
        print('lol')
        model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics='accuracy')
        print('lol')
        history = model.fit([enc_input_train,dec_input_train], dec_target_train, 
                            epochs=config.epochs,
                            validation_data = ([enc_input_dev,dec_input_dev], dec_target_dev),
                            callbacks = [WandbCallback()]
                            )
        print('lol')
        name = str(config.encoder_block) + '_' + str(config.encoder_layers) + '_' + str(config.decoder_block) + '_' + str(config.decoder_layers) + '_' + str(config.hidden_layer) + '_' + str(config.dropout) + '_' + str(config.epochs)
        location = '/content/drive/MyDrive/Transliteration/' + name
        model.save(location)

{'method': 'grid',
 'parameters': {'decoder_block': {'values': ['LSTM']},
                'dropout': {'values': [0]},
                'encoder_block': {'values': ['LSTM']},
                'epochs': {'values': [60]},
                'hidden_layer': {'values': [128]},
                'latent_dimension': {'values': [256]}}}


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="CS6910-assg3")
wandb.agent(sweep_id, training_sweep)